In [ ]:
%load_ext autoreload
%autoreload 2

from research_lib.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
from weight_estimation.utils import get_left_right_keypoint_arrs, convert_to_world_point_arr, CameraMetadata
from sift_matching import generate_point_correspondences
from matplotlib import pyplot as plt
import cv2
import numpy as np


In [ ]:
s3 = S3AccessUtils('/root/data')

<h1> Load sample data </h1>

In [ ]:
rds = RDSAccessUtils()
query = """
    SELECT * 
    FROM 
        prod.biomass_computations
    WHERE
        pen_id = 88 AND
        group_id = '88' AND
        akpd_score = 0
    LIMIT 20;
"""

df = rds.extract_from_database(query)

<h1> Generate sample 3D point cloud </h1>

In [ ]:
idx = 7
row = df.iloc[idx]

left_crop_url, right_crop_url, ann, camera_metadata = row.left_crop_url, row.right_crop_url, row.annotation, row.camera_metadata
cm = CameraMetadata(
    focal_length=camera_metadata['focalLength'],
    focal_length_pixel=camera_metadata['focalLengthPixel'],
    baseline_m=camera_metadata['baseline'],
    pixel_count_width=camera_metadata['pixelCountWidth'],
    pixel_count_height=camera_metadata['pixelCountHeight'],
    image_sensor_width=camera_metadata['imageSensorWidth'],
    image_sensor_height=camera_metadata['imageSensorHeight']
)

left_points_sift, right_points_sift, left_corner, right_corner = generate_point_correspondences(left_crop_url, right_crop_url, ann)
X_left, X_right = get_left_right_keypoint_arrs(ann)
# left_points = np.vstack([left_points_sift, X_left])
# right_points = np.vstack([right_points_sift, X_right])
left_points = left_points_sift
right_points = right_points_sift

point_cloud = convert_to_world_point_arr(left_points, right_points, cm)
rgbd = np.array([left_points[:, 0] - left_corner[0], left_points[:, 1] - left_corner[1], point_cloud[:, 1]]).T


In [ ]:
left_crop_f, _, _ = s3.download_from_url(left_crop_url)

<h1> Scratch work </h1>

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.tri as tri
from scipy.spatial import Delaunay

In [ ]:
fig1, ax1 = plt.subplots(figsize=(20, 10))

points = rgbd #[np.random.choice(rgbd.shape[0], 100)]
triang = tri.Triangulation(points[:, 0], points[:, 1])
ax1.set_aspect('equal')
tpc = ax1.tripcolor(triang, points[:, 2], shading='flat')
fig1.colorbar(tpc)
plt.imshow(cv2.imread(left_crop_f))
plt.gca().set_aspect('equal', adjustable='box')
ax1.set_title('tripcolor of Delaunay triangulation, flat shading')


In [ ]:
s3 = S3AccessUtils('/root/data')
left_crop_f, _, _ = s3.download_from_url(left_crop_url)

In [ ]:
cv2.imread(left_crop_f).shape

In [ ]:
plt.figure(figsize=(20, 10))
plt.imshow(cv2.imread(left_crop_f))
plt.show()